# Protected Area Boundary Change

This notebook outlines the methodology used to measure at protected area boundaries via GEE. 

The notebook queries MODIS imagery and returns the gradient values of relevant bands as well as the vegetation indices NDVI and EVI. This code expects an annual time span and returns a geoTIFF for each band in each park for that year.

In [1]:
import sys
sys.path.append('../src')
from utils import *
from config import *
import pandas as pd
from datetime import datetime

In [2]:
import ee
ee.Authenticate()
ee.Initialize(project='dse-staff')
print(ee.String('Hello from the Earth Engine servers!').getInfo())

Hello from the Earth Engine servers!


## Class Definitions

In [ ]:
def main(wdpaid, year): #, show_map=False, band_name=None):
    """Main function to process protected area boundary analysis"""
    # Initialize classes
    geo_ops = GeometryOperations()
    img_ops = ImageOperations()
    stats_ops = StatsOperations()
    #viz = Visualization()
    feature_processor = FeatureProcessor(geo_ops, img_ops, stats_ops)
    #exporter = ExportResults()

    # Load and process protected area geometry
    pa = load_local_data(wdpaid)
    pa_geometry = pa.geometry()
    aoi = geo_ops.buffer_polygon(pa_geometry)
    aoi = geo_ops.mask_water(aoi)
    aoi_with_biome = geo_ops.get_biome(aoi)

    # Process imagery and add indices
    modis_ic = img_ops.modis.filter(img_ops.filter_for_year(aoi, year))
    band_names = modis_ic.first().bandNames()
    composite = modis_ic.reduce(ee.Reducer.median()).rename(band_names).clip(aoi)
    image = img_ops.add_indices_to_image(composite)

    # Process features and collect statistics
    feature_info = feature_processor.collect_feature_info(pa, aoi_with_biome)
    #computed_stats = feature_processor.process_all_bands(image, pa_geometry, aoi)
    #all_stats = feature_processor.compile_statistics(feature_info, computed_stats, year)
    
    # Save results
    #df = pd.DataFrame(all_stats)
    #exporter.save_df_to_gcs(df, 'dse-staff', wdpaid, year)

    # Visualization
    #if show_map:
    #    band_stats = next(cs for cs in computed_stats if cs["band_name"] == band_name)
    #    Map = viz.create_map(pa_geometry, band_stats['buffer_pixels'], band_stats['boundary_pixels'])
    #    return Map
    #return print("Analysis complete for WDPA ID:", wdpaid, "for the year:", year)

    # ... previous code ...
    feature_info = feature_processor.collect_feature_info(pa, aoi_with_biome)
    features = feature_processor.process_all_bands_ee(image, pa_geometry, aoi, feature_info, year)
    stats_fc = ee.FeatureCollection(features)

    task = ee.batch.Export.table.toCloudStorage(
        collection=stats_fc,
        description=f'gradient_stats_export_{wdpaid}_{year}',
        bucket='dse-staff',
        fileNamePrefix=f'protected_areas/tables/gradient_stats_{wdpaid}_{year}',
        fileFormat='CSV'
    )
    task.start()
    print(f"Export task started for {wdpaid}, {year}")

    # Export each band as an image
    for band_name in image.bandNames().getInfo():
        band_image = image.select(band_name).clip(aoi)
        img_task = ee.batch.Export.image.toCloudStorage(
            image=band_image,
            description=f'{band_name}_export_{wdpaid}_{year}',
            bucket='dse-staff',
            fileNamePrefix=f'protected_areas/images/{band_name}_{wdpaid}_{year}',
            fileFormat='GeoTIFF',
            scale=500,
            maxPixels=1e10
        )
        img_task.start()
        print(f"Export image task started for {band_name}, {wdpaid}, {year}")

In [4]:
main("916", 2010) #, show_map=True, band_name="EVI")

/usr/local/lib/python3.10/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for JRC/GSW1_0/GlobalSurfaceWater! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/JRC_GSW1_0_GlobalSurfaceWater

  warnings.warn(warning, category=DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for MODIS/006/MOD09A1! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD09A1

  warnings.warn(warning, category=DeprecationWarning)


Export task started for 916, 2010
Export image task started for sur_refl_b01, 916, 2010
Export image task started for sur_refl_b02, 916, 2010
Export image task started for sur_refl_b03, 916, 2010
Export image task started for sur_refl_b04, 916, 2010
Export image task started for sur_refl_b05, 916, 2010
Export image task started for sur_refl_b06, 916, 2010
Export image task started for sur_refl_b07, 916, 2010
Export image task started for QA, 916, 2010
Export image task started for SolarZenith, 916, 2010
Export image task started for ViewZenith, 916, 2010
Export image task started for RelativeAzimuth, 916, 2010
Export image task started for StateQA, 916, 2010
Export image task started for DayOfYear, 916, 2010
Export image task started for EVI, 916, 2010
Export image task started for NDVI, 916, 2010
